# Face detection using Amazon Rekognition

***
This notebook provides a walkthrough of [face detection API](https://docs.aws.amazon.com/rekognition/latest/dg/faces.html) in Amazon Rekognition to identify faces.
***

In [ ]:
# Initialize dependencies
import boto3
import botocore
from IPython.display import HTML, display, Image as IImage
import time

# Initialize clients
REGION = boto3.session.Session().region_name
rekognition = boto3.client('rekognition', REGION)
s3 = boto3.client('s3', REGION)

%store -r bucket_name

# Detect faces in image
***

In [ ]:
# Show image
image_name = "media/looking_at_screen.jpg"
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': image_name})))

#### Call Rekognition to detect faces in the image

<https://docs.aws.amazon.com/rekognition/latest/dg/faces-detect-images.html>

In [ ]:
detect_faces_response = rekognition.detect_faces(
    Attributes=['ALL'],
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': image_name,
        }
    }
)

#### Review the raw JSON reponse from Rekognition

In the JSON response below, you will see faces, detected attributes, confidence score and additional information.

In [ ]:
display(detect_faces_response)

#### Display number of faces detected

In [ ]:
print("Number of faces detected: {}".format(len(detect_faces_response['FaceDetails'])))

# Recognize faces in video
Face recognition in video is an async operation. 
https://docs.aws.amazon.com/rekognition/latest/dg/faces-sqs-video.html. 

- First we start a face detection job which returns a Job Id.
- We can then call `get_face_detection` to get the job status and after job is complete, we can get object metadata.
- In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.
***

#### Show video in the player

In [ ]:
video_name = "media/leaving.mp4"
s3_video_url = s3.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': video_name})

video_tag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(s3_video_url)
video_ui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(video_tag)
display(HTML(video_ui))


#### Call Rekognition to start a job for face detection

In [ ]:
start_face_detection = rekognition.start_face_detection(
    Video={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': video_name,
        }
    },
)

faces_job_id = start_face_detection['JobId']
display("Job Id: {0}".format(faces_job_id))

### Additional (Optional) Request Attributes

ClientRequestToken, JobTag, MinConfidence, and NotificationChannel:
https://docs.aws.amazon.com/rekognition/latest/APIReference/API_StartFaceDetection.html

FaceDetail:
https://docs.aws.amazon.com/rekognition/latest/APIReference/API_FaceDetail.html


#### Wait for object detection job to complete

In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.

In [ ]:
get_face_detection = rekognition.get_face_detection(
    JobId=faces_job_id
)

while(get_face_detection['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    get_face_detection = rekognition.get_face_detection(
        JobId=faces_job_id)
    
display(get_face_detection['JobStatus'])

#### Review raw JSON reponse from Rekognition

In the JSON response below, you will see list of detected faces and attributes.
For each detected face, you will see information like Timestamp

In [ ]:
display(get_face_detection)

#### Display face detected by timestamp and alert when faces are not detected

In [ ]:
# Faces detected in each frame
prev_ts = 0
threshold = 1000 # ms
for face in get_face_detection['Faces']:
    ts = face["Timestamp"]
    cconfidence = face["Face"]["Confidence"]
    if ts-prev_ts>threshold:
        print("ALERT - no face detected for {} seconds".format((ts-prev_ts)/1000))
    print("Detected face on Timestamp: {} with confidence: {}".format(ts, cconfidence))
    prev_ts = ts

***
### References
- https://docs.aws.amazon.com/rekognition/latest/APIReference/API_DetectFaces.html
- https://docs.aws.amazon.com/rekognition/latest/APIReference/API_StartFaceDetection.html
- https://docs.aws.amazon.com/rekognition/latest/APIReference/API_GetFaceDetection.html

***

You have successfully used Amazon Rekognition to detect faces in images and videos.